## 2. Scanning with SARUS

In [ ]:
import os
import re
import sys
import argparse
import subprocess
import matplotlib
import time
import random
import string
import shlex
import shutil
import glob
import pickle
import csv
import operator
import joblib
import pandas as pd
import numpy as np
from itertools import groupby
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from collections import defaultdict
from sklearn.ensemble import BaggingClassifier
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from Bio import SeqIO
from sklearn.feature_selection import RFE
from sklearn import preprocessing
import pybedtools as pbt
import pyBigWig as pbw
from datetime import date
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from itertools import chain
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from sklearn.ensemble import AdaBoostClassifier
from adjustText import adjust_text
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from pylab import *
%matplotlib inline

### Scanning GHTS data

In [ ]:
root = "[your path to dir]/MEX-ArChIPelago/" 
basicdir = os.path.abspath('GHTS/')
outputdir = os.path.abspath('GHTS/outputdir')
train_dir = os.path.abspath('GHTS/Train/') 
test_dir = os.path.abspath('GHTS/Test/') 

if not os.path.exists(basicdir): 
    os.makedirs(basicdir)
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)


TFs_CHS_AFS = pd.read_csv("./Input_data/best_20_motif_CHS_GHTS.txt", sep="\t", header=None)

In [ ]:
seed(42)
model = "RandomForestClassifier"
organism = "HUMAN_GHTS"
sarsus_home = "" # if you wish to use a specific release, please specify ./sarus/releases/sarus-x.x.x.jar
pwmdir_mono = "./Input_data/best_20_motif_CHS_GHTS/GHTS"

TF_dir = train_dir

print(" ")
сalc = 0
for TF in sorted(TFs_CHS_AFS):  # For DEMO tests use ["GABPA"] instead of TFs_CHS_AFS. 
    print(TF)
    print("TF # {сalc} of {all_c}".format(all_c=len(TFs_CHS_AFS), сalc=сalc))
    сalc += 1

    print("Dirname is:", TF)

    new_dir_name = f"{TF_dir}/{TF}/pwm_scanning_results_addshift"
    print("New_dir_name", new_dir_name)

    if not os.path.exists(new_dir_name):
        os.makedirs(new_dir_name)

    os.chdir(new_dir_name)
    dir_list = os.listdir(pwmdir_mono + "/" + TF)

    print("dir_list", len(dir_list))
    for exp_pwm_dir in dir_list:
        pwm_local_dir = pwmdir_mono + "/" + TF + "/" + exp_pwm_dir
        pwm_local_list = [x for x in os.listdir(pwm_local_dir) if "pwm" in x]
        print("Sarus is running ...")
        for matrix in pwm_local_list:

            line = 'java -Xmx2G -cp {sarsus_home} ru.autosome.SARUS {mfa_file} {matrix} --skipn --show-non-matching --output-scoring-mode score besthit | grep -v \> > {out_file}'.format(sarsus_home = root + "/sarus/releases/sarus-2.0.1.jar",
            mfa_file = '/'.join([TF_dir, TF, "positives_no_NF.fa"]),
            matrix = '/'.join([pwm_local_dir, matrix]),
            out_file = '/'.join([new_dir_name, ".".join(matrix.split(".")[:-1]) + "_positives.tab"]))

            p = subprocess.Popen(line, shell=True)
            p.wait()
                     
            !rm tmp.sh

            line1 = "awk -F. \'{print $1\".\"substr($2,1,2)}\' " + ".".join(matrix.split(".")[:-1]) + "_positives.tab" + " > " + ".".join(matrix.split(".")[:-1]) + "_positives_cut.tab"
            line2 = "rm " + ".".join(matrix.split(".")[:-1]) + "_positives.tab"

            with open("tmp.sh", "w") as f:
                f.write(line1)
                f.write(" ; ")
                f.write(line2)
            !bash tmp.sh

            line = 'java -Xmx2G -cp {sarsus_home} ru.autosome.SARUS {mfa_file} {matrix} --skipn --show-non-matching --output-scoring-mode score besthit | grep -v \> > {out_file}'.format(sarsus_home = root + "/sarus/releases/sarus-2.0.1.jar",
            mfa_file = '/'.join([TF_dir, TF, "foreigns_no_NF.fa"]),
            matrix = '/'.join([pwm_local_dir, matrix]),
            out_file = '/'.join([new_dir_name, ".".join(matrix.split(".")[:-1]) + "_foreigns.tab"]))

            p = subprocess.Popen(line, shell=True)
            p.wait()
            
            !rm tmp.sh

            line1 = "awk -F. \'{print $1\".\"substr($2,1,2)}\' " + ".".join(matrix.split(".")[:-1]) + "_foreigns.tab" + " > " + ".".join(matrix.split(".")[:-1]) + "_foreigns_cut.tab"
            line2 = "rm " + ".".join(matrix.split(".")[:-1]) + "_foreigns.tab"

            with open("tmp.sh", "w") as f:
                f.write(line1)
                f.write(" ; ")
                f.write(line2)
            !bash tmp.sh
            !rm tmp.sh
            
            line = 'java -Xmx2G -cp {sarsus_home} ru.autosome.SARUS {mfa_file} {matrix} --skipn --show-non-matching --output-scoring-mode score besthit | grep -v \> > {out_file}'.format(sarsus_home = root + "/sarus/releases/sarus-2.0.1.jar",
            mfa_file = '/'.join([TF_dir, TF, "random_no_NF.fa"]),
            matrix = '/'.join([pwm_local_dir, matrix]),
            out_file = '/'.join([new_dir_name, ".".join(matrix.split(".")[:-1]) + "_random.tab"]))
            #print(line)

            p = subprocess.Popen(line, shell=True)
            p.wait()
                 
            !rm tmp.sh

            line1 = "awk -F. \'{print $1\".\"substr($2,1,2)}\' " + ".".join(matrix.split(".")[:-1]) + "_random.tab" + " > " + ".".join(matrix.split(".")[:-1]) + "_random_cut.tab"
            line2 = "rm " + ".".join(matrix.split(".")[:-1]) + "_random.tab"

            with open("tmp.sh", "w") as f:
                f.write(line1)
                f.write(" ; ")
                f.write(line2)
            !bash tmp.sh
            !rm tmp.sh

### Scanning CHS data

In [ ]:
basicdir = os.path.abspath('CHS/')
outputdir = os.path.abspath('CHS/outputdir')
train_dir = os.path.abspath('CHS/Train/') 
test_dir = os.path.abspath('CHS/Test/') 

if not os.path.exists(basicdir): 
    os.makedirs(basicdir)
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

In [ ]:
seed(42)
model = "RandomForestClassifier"
organism = "HUMAN_CHS"
sarsus_home = "" # if you wish to use a specific release, please specify ./sarus/releases/sarus-x.x.x.jar
pwmdir_mono = "./Input_data/best_20_motif_CHS_GHTS/CHS"

TF_dir = train_dir

print(" ")
сalc = 0
for TF in sorted(TFs_CHS_AFS):  # For DEMO tests use ["GABPA"] instead of TFs_CHS_AFS. 
    print(TF)
    print("TF # {сalc} of {all_c}".format(all_c=len(TFs_CHS_AFS), сalc=сalc))
    сalc += 1

    print("Dirname is:", TF)

    new_dir_name = f"{TF_dir}/{TF}/pwm_scanning_results_addshift"
    print("New_dir_name", new_dir_name)

    if not os.path.exists(new_dir_name):
        os.makedirs(new_dir_name)

    os.chdir(new_dir_name)
    dir_list = os.listdir(pwmdir_mono + "/" + TF)

    print("dir_list", len(dir_list))
    for exp_pwm_dir in dir_list:
        pwm_local_dir = pwmdir_mono + "/" + TF + "/" + exp_pwm_dir
        pwm_local_list = [x for x in os.listdir(pwm_local_dir) if "pwm" in x]
        print("Sarus is running ...")
        for matrix in pwm_local_list:

            line = 'java -Xmx2G -cp {sarsus_home} ru.autosome.SARUS {mfa_file} {matrix} --skipn --show-non-matching --output-scoring-mode score besthit | grep -v \> > {out_file}'.format(sarsus_home = root + "/sarus/releases/sarus-2.0.1.jar",
            mfa_file = '/'.join([TF_dir, TF, "positives_no_NF.fa"]),
            matrix = '/'.join([pwm_local_dir, matrix]),
            out_file = '/'.join([new_dir_name, ".".join(matrix.split(".")[:-1]) + "_positives.tab"]))

            p = subprocess.Popen(line, shell=True)
            p.wait()
                     
            !rm tmp.sh

            line1 = "awk -F. \'{print $1\".\"substr($2,1,2)}\' " + ".".join(matrix.split(".")[:-1]) + "_positives.tab" + " > " + ".".join(matrix.split(".")[:-1]) + "_positives_cut.tab"
            line2 = "rm " + ".".join(matrix.split(".")[:-1]) + "_positives.tab"

            with open("tmp.sh", "w") as f:
                f.write(line1)
                f.write(" ; ")
                f.write(line2)
            !bash tmp.sh

            line = 'java -Xmx2G -cp {sarsus_home} ru.autosome.SARUS {mfa_file} {matrix} --skipn --show-non-matching --output-scoring-mode score besthit | grep -v \> > {out_file}'.format(sarsus_home = root + "/sarus/releases/sarus-2.0.1.jar",
            mfa_file = '/'.join([TF_dir, TF, "foreigns_no_NF.fa"]),
            matrix = '/'.join([pwm_local_dir, matrix]),
            out_file = '/'.join([new_dir_name, ".".join(matrix.split(".")[:-1]) + "_foreigns.tab"]))

            p = subprocess.Popen(line, shell=True)
            p.wait()
            
            !rm tmp.sh

            line1 = "awk -F. \'{print $1\".\"substr($2,1,2)}\' " + ".".join(matrix.split(".")[:-1]) + "_foreigns.tab" + " > " + ".".join(matrix.split(".")[:-1]) + "_foreigns_cut.tab"
            line2 = "rm " + ".".join(matrix.split(".")[:-1]) + "_foreigns.tab"

            with open("tmp.sh", "w") as f:
                f.write(line1)
                f.write(" ; ")
                f.write(line2)
            !bash tmp.sh
            !rm tmp.sh
            
            line = 'java -Xmx2G -cp {sarsus_home} ru.autosome.SARUS {mfa_file} {matrix} --skipn --show-non-matching --output-scoring-mode score besthit | grep -v \> > {out_file}'.format(sarsus_home = root + "/sarus/releases/sarus-2.0.1.jar",
            mfa_file = '/'.join([TF_dir, TF, "random_no_NF.fa"]),
            matrix = '/'.join([pwm_local_dir, matrix]),
            out_file = '/'.join([new_dir_name, ".".join(matrix.split(".")[:-1]) + "_random.tab"]))
            #print(line)

            p = subprocess.Popen(line, shell=True)
            p.wait()
                 
            !rm tmp.sh

            line1 = "awk -F. \'{print $1\".\"substr($2,1,2)}\' " + ".".join(matrix.split(".")[:-1]) + "_random.tab" + " > " + ".".join(matrix.split(".")[:-1]) + "_random_cut.tab"
            line2 = "rm " + ".".join(matrix.split(".")[:-1]) + "_random.tab"

            with open("tmp.sh", "w") as f:
                f.write(line1)
                f.write(" ; ")
                f.write(line2)
            !bash tmp.sh
            !rm tmp.sh